## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [ ]:
import numpy as np
from numba import njit
import pandas as pd
import random

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
%%time
a = []
for i in range(1000000):
    k = random.randint(0,1000)
    a.append(k)
random.shuffle(a)
b = [j+100 for j in a]
#print(a[:15])
#print(b[:15])
print(sum(b)//len(b))

600
CPU times: user 2.35 s, sys: 11.7 ms, total: 2.36 s
Wall time: 2.42 s


In [ ]:
def task_1():
  a = []
  for i in range(1000000):
    k = random.randint(0,1000)
    a.append(k)
  random.shuffle(a)
  b = [j+100 for j in a]
#print(a[:15])
#print(b[:15])
  return print(sum(b)//len(b))
task_1()

599


In [ ]:
%time task_1() 

600
CPU times: user 2.1 s, sys: 89.6 ms, total: 2.19 s
Wall time: 2.22 s


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
a = a+a
b = b+b
print(len(a), len(b))

2000000 2000000


In [ ]:
import string

df = pd.DataFrame({'col1':a, 'col2':b, 'col3':a[::-1], 'col4':b[::-1]})
randomLetter = random.choice(string.ascii_letters)

df['key'] = [random.choice(string.ascii_letters) for i in range(2000000)]
df

,col1,col2,col3,col4,key
0,34,134,357,457,K
1,123,223,459,559,j
2,722,822,378,478,p
3,615,715,838,938,q
4,543,643,684,784,W
...,...,...,...,...,...
1999995,684,784,543,643,c
1999996,838,938,615,715,M
1999997,378,478,722,822,V
1999998,459,559,123,223,y


In [ ]:
s = ['a','b','c','d','e']
df[df['key'].isin(s)]

,col1,col2,col3,col4,key
1,454,554,191,291,a
5,228,328,277,377,d
10,206,306,859,959,b
19,271,371,152,252,b
21,76,176,977,1077,b
...,...,...,...,...,...
1999982,323,423,60,160,d
1999983,873,973,497,597,e
1999984,296,396,698,798,b
1999985,524,624,913,1013,e


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 11.2 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
%load_ext line_profiler

In [ ]:
recipes = pd.read_csv('/content/recipes_sample.csv', delimiter = ',')
reviews = pd.read_csv('/content/reviews_sample.csv', delimiter = ',')
reviews['date'] = pd.to_datetime(reviews['date'], format="%Y-%m-%d")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126696 entries, 0 to 126695
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  126696 non-null  int64         
 1   user_id     126696 non-null  int64         
 2   recipe_id   126696 non-null  int64         
 3   date        126696 non-null  datetime64[ns]
 4   rating      126696 non-null  int64         
 5   review      126679 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 5.8+ MB


In [ ]:
def r1():
    sums = 0
    cnt = 0
    for i in reviews.iterrows():
        if i[1].date.year == 2010:
            sums += i[1].rating
            cnt += 1
    return sums/cnt
r1()

4.4544402182900615

In [ ]:
%time r1()

CPU times: user 7.02 s, sys: 62.4 ms, total: 7.09 s
Wall time: 7.52 s


4.4544402182900615

In [ ]:
def r2():
    sums = 0
    cnt = 0
    for i in reviews[reviews['date'].dt.year == 2010].iterrows():
        sums += i[1].rating
        cnt += 1
    return sums/cnt
r2()

4.4544402182900615

In [ ]:
%time r2()

CPU times: user 625 ms, sys: 1.67 ms, total: 627 ms
Wall time: 631 ms


4.4544402182900615

In [ ]:
def r3():
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()
r3()

4.4544402182900615

In [ ]:
%time r3()

CPU times: user 22.3 ms, sys: 0 ns, total: 22.3 ms
Wall time: 26.7 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
%lprun -f r1 r1()

Timer unit: 1e-09 s

Total time: 13.3736 s
File: <ipython-input-18-1b2eb7000e32>
Function: r1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def r1():
     2         1       1775.0   1775.0      0.0      sums = 0
     3         1        208.0    208.0      0.0      cnt = 0
     4    126696 10721012698.0  84620.0     80.2      for i in reviews.iterrows():
     5    114602 2440344315.0  21294.1     18.2          if i[1].date.year == 2010:
     6     12094  206573428.0  17080.7      1.5              sums += i[1].rating
     7     12094    5691578.0    470.6      0.0              cnt += 1
     8         1       1183.0   1183.0      0.0      return sums/cnt



In [ ]:
%lprun -f r2 r2()

Timer unit: 1e-09 s

Total time: 1.15113 s
File: <ipython-input-20-02fc350dbfac>
Function: r2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def r2():
     2         1       1235.0   1235.0      0.0      sums = 0
     3         1        217.0    217.0      0.0      cnt = 0
     4     12094  916350002.0  75769.0     79.6      for i in reviews[reviews['date'].dt.year == 2010].iterrows():
     5     12094  229287402.0  18958.8     19.9          sums += i[1].rating
     6     12094    5487951.0    453.8      0.5          cnt += 1
     7         1       1373.0   1373.0      0.0      return sums/cnt

In [ ]:
%lprun -f r3 r3()

Timer unit: 1e-09 s

Total time: 0.0228886 s
File: <ipython-input-22-1cd2dda05203>
Function: r3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def r3():
     2         1   22888627.0 22888627.0    100.0      return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
%%time
result_origin = get_word_reviews_count(reviews)

CPU times: user 21.4 s, sys: 197 ms, total: 21.6 s
Wall time: 22.5 s


In [ ]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-09 s

Total time: 46.0454 s
File: <ipython-input-27-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def get_word_reviews_count(df):
     2         1       1127.0   1127.0      0.0      word_reviews = {}
     3    126679 12137726314.0  95814.8     26.4      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 2752607743.0  21729.0      6.0          recipe_id, review = row['recipe_id'], row['review']
     5    126679  645602640.0   5096.4      1.4          words = review.split(' ')
     6   6792010 1710117072.0    251.8      3.7          for word in words:
     7   6617066 3154917310.0    476.8      6.9              if word not in word_reviews:
     8    174944   92776360.0    530.3      0.2                  word_reviews[word] = []
     9   6792010 4348588787.0    640.3      9.4              word_reviews[word].append(recipe_id)
    10                                               
    11         1        945.0    945.0      0.0      word_reviews_count = {}
    12    126679 11360276764.0  89677.7     24.7      for _, row in df.dropna(subset=['review']).iterrows():
    13    126679 1672386976.0  13201.8      3.6          review = row['review']

In [ ]:
def get_word_reviews_count_opt(df):
    word_reviews_count = {}
    for row in df.dropna(subset=['review'])['review'].str.split(' '):
        for word in row:
            if word in word_reviews_count:
                word_reviews_count[word] += 1
            else:
                word_reviews_count[word] = 1
    return word_reviews_count

In [ ]:
%%time
result_opt = get_word_reviews_count_opt(reviews)

CPU times: user 2.82 s, sys: 534 ms, total: 3.36 s
Wall time: 3.37 s


In [ ]:
%lprun -f get_word_reviews_count_opt get_word_reviews_count_opt(reviews)

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
#1 способ
def MAPE_1(rev: pd.DataFrame):
    rating_clear = rev[rev['rating'] != 0]['rating']
    mean = rating_clear.mean()
    return rating_clear.apply(lambda x: abs(mean - x)).sum() / mean * 100 / rating_clear.count()

In [ ]:
%time MAPE_1(reviews)

CPU times: user 81.7 ms, sys: 0 ns, total: 81.7 ms
Wall time: 86.8 ms


11.079150232267242

In [ ]:
#2 способ
@njit
def MAPE_2(rev: pd.DataFrame):
    rating_clear = rev[rev['rating'] != 0]['rating']
    mean = rating_clear.mean()
    return rating_clear.apply(lambda x: abs(mean - x)).sum() / mean * 100 / rating_clear.count()

In [ ]:
#3 способ
def MAPE_3(rev: pd.DataFrame):
    rating_clear = rev[rev['rating'] != 0]['rating'].to_numpy()
    mean = rating_clear.mean()
    return np.absolute(rating_clear - mean).sum() / mean * 100 / rating_clear.shape[0]

In [ ]:
%time MAPE_3(reviews)

CPU times: user 18 ms, sys: 1 ms, total: 19 ms
Wall time: 22.8 ms


11.079150232267242

In [ ]:
#4 способ
@njit
def MAPE_4_0(a: np.array):
    mean = a.mean()
    return np.absolute(a - mean).sum() / mean * 100 / a.shape[0]

In [ ]:
def MAPE_4(rev: pd.DataFrame):
    rating_clear = rev[rev['rating'] != 0]['rating'].to_numpy()
    return MAPE_4_0(rating_clear)

In [ ]:
%time MAPE_4(reviews)

CPU times: user 1.03 s, sys: 127 ms, total: 1.16 s
Wall time: 1.09 s


11.079150232261632